In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.modeling import models, fitting
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data
from functions.plot_angle_gradient import plot_angle_gradient

In [ ]:
# Open data files
from functions.load_pickles import load_pickles
stokes_list, _ = load_pickles(select="stokes")

In [ ]:
# Extract each Stokes parameter into dictionary, to make it easier to work with
I = stokes_list['I']
Q = stokes_list['Q']
U = stokes_list['U']
V = stokes_list['V']

In [ ]:
lambda0  = [6301.51, 6302.50]   # Angstroms to cm

In [ ]:
# Fit a Gaussian to the data and find its minimum
verbose = 0

shape = np.shape(I.data_n[:,:,:2])
fit_quality = np.zeros(shape)
min_x_array = np.zeros(shape)
dl = np.zeros(shape)

for i in range(0, shape[0]):
    if np.mod(i,50) == 0:
        print(f'Row {i} of {np.shape(I.data_n)[0]}')
    for j in range(0, shape[1]):
        x = I.wave_array[:60]
        y = I.data_n[i,j,:60]
        # Normalize y for fitting stability
        y_norm = (y - np.median(y)) / np.abs(y).max()
        # Initial guess: amplitude, mean, stddev
        amplitude_guess = y_norm.min()
        mean_guess = x[np.argmin(y_norm)]
        stddev_guess = (x.max() - x.min()) / 8
        init = models.Gaussian1D(amplitude=amplitude_guess, mean=mean_guess, stddev=stddev_guess)
        fitter = fitting.LevMarLSQFitter()
        fit = fitter(init, x, y_norm)
        min_x = fit.mean.value
        min_x_array[i, j, 0] = min_x
        dl[i, j, 0] = min_x - lambda0[0]
        min_y = fit(min_x) * np.abs(y).max() + np.median(y)  # convert back to original scale
        # Fit quality: normalized RMSE
        residual = y - (fit(x) * np.abs(y).max() + np.median(y))
        rmse = np.sqrt(np.mean(residual**2))
        norm_rmse = rmse / (np.abs(y).max() - np.abs(y).min() + 1e-8)
        fit_quality[i, j, 0] = norm_rmse
        if (verbose) or (fit_quality[i,j,0] > 0.2):
            print(f"Minimum of Gaussian fit: x = {min_x}, fit quality = {norm_rmse:.4f}")
            print(f"Distance: {dl[i, j, 0]}")
            plt.plot(x, y, label='Data')
            plt.plot(x, fit(x) * np.abs(y).max() + np.median(y), label='Gaussian fit', color='orange')
            plt.axvline(min_x, color='green', linestyle='--', label='Fit minimum')
            plt.axvline(lambda0[0], color='purple', linestyle='--', label='Reference line')
            plt.legend()
            plt.title(f'Pixel ({i},{j})')
            plt.show()

In [ ]:
plot_data(dl[:,:,0], title=r"$\Delta \lambda $, in $\mathrm{\AA}$", colourmap='bwr', scale=[-0.08, 0.08])

In [ ]:
plot_data(fit_quality[:,:,0], title="RMSW", colourmap='grey')

In [ ]:
c = 2.99792458e5  # [km · s−1]

# Rebecca Centeno, equation 3
v = c * dl[:,:,0] / lambda0[0]

plot_data(v, title=f"Velocity [km/s]", colourmap='bwr', scale=[-3, 3])
plot_data(v -v.mean(), title=f"Velocity [km/s]", colourmap='bwr', scale=[-3, 3])

In [ ]:
v.mean()

In [ ]:
# Save min_x_array, fit_quality, and dl as pickle files
import pickle
with open('generated/min_x_array.pickle', 'wb') as f:
    pickle.dump(min_x_array, f)
with open('generated/fit_quality.pickle', 'wb') as f:
    pickle.dump(fit_quality, f)
with open('generated/dl.pickle', 'wb') as f:
    pickle.dump(dl, f)
with open('generated/velocity.pickle', 'wb') as f:
    pickle.dump(v, f)
print('Saved min_x_array, fit_quality, velocity, and dl to generated/ as pickle files')
